## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-04-20-50-52 +0000,nyt,"In New Jersey, voters say Trump influenced the...",https://www.nytimes.com/live/2025/11/04/us/ele...
1,2025-11-04-20-47-05 +0000,nyt,Dick Cheney’s Unwavering Image,https://www.nytimes.com/2025/11/04/style/dick-...
2,2025-11-04-20-44-00 +0000,wsj,Apollo Invests $75 Billion in Quarter As Firm ...,https://www.wsj.com/articles/apollo-invests-75...
3,2025-11-04-20-43-06 +0000,startribune,Talks to end the government shutdown intensify...,https://www.startribune.com/talks-to-end-the-g...
4,2025-11-04-20-32-33 +0000,wapo,AI stocks waver as ‘Big Short’ investor bets a...,https://www.washingtonpost.com/business/2025/1...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2337/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,44
0,new,19
225,will,16
89,election,15
10,cheney,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
243,2025-11-04-10-00-00 +0000,wsj,When the Supreme Court hears arguments over Pr...,https://www.wsj.com/politics/policy/whats-at-s...,124
28,2025-11-04-19-43-00 +0000,wsj,The Trump administration will comply with a co...,https://www.wsj.com/politics/policy/trump-thre...,111
245,2025-11-04-09-30-53 +0000,bbc,Trump backs Cuomo for New York City mayor and ...,https://www.bbc.com/news/articles/c231e284345o...,104
111,2025-11-04-17-31-00 +0000,wsj,Some Tuesday elections will test whether Trump...,https://www.wsj.com/politics/elections/divided...,103
70,2025-11-04-18-37-45 +0000,latimes,Trump pushes hard against Mamdani as New Yorke...,https://www.latimes.com/politics/story/2025-11...,96


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
243,124,2025-11-04-10-00-00 +0000,wsj,When the Supreme Court hears arguments over Pr...,https://www.wsj.com/politics/policy/whats-at-s...
95,71,2025-11-04-17-56-00 +0000,wsj,The government pays out $100 billion a year in...,https://www.wsj.com/politics/policy/what-to-kn...
147,64,2025-11-04-15-38-07 +0000,nyt,Election Day Bomb Threats Briefly Close Some P...,https://www.nytimes.com/2025/11/04/us/politics...
201,59,2025-11-04-11-52-00 +0000,wsj,"Dick Cheney, Powerful Former Vice President Wh...",https://www.wsj.com/politics/dick-cheney-power...
119,47,2025-11-04-17-18-25 +0000,cbc,"NYC mayor's race draws attention, but Californ...",https://www.cbc.ca/news/world/us-tuesday-off-y...
156,43,2025-11-04-15-01-47 +0000,nypost,France threatens to block Shein over sale of c...,https://nypost.com/2025/11/04/world-news/franc...
111,39,2025-11-04-17-31-00 +0000,wsj,Some Tuesday elections will test whether Trump...,https://www.wsj.com/politics/elections/divided...
37,38,2025-11-04-19-26-14 +0000,nypost,Typhoon Kalmaegi leaves at least 26 dead in Ph...,https://nypost.com/2025/11/04/world-news/typho...
225,38,2025-11-04-11-00-00 +0000,latimes,Elon Musk's latest Tesla pay package could mak...,https://www.latimes.com/business/story/2025-11...
235,37,2025-11-04-10-23-31 +0000,bbc,'Taxes are going up' - BBC decodes Reeves's pr...,https://www.bbc.com/news/videos/cpd2yx6v687o?a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
